In [ ]:
import BiomarkerMediumRelation_Extraction


import cPickle, os, sys, matplotlib

sys.path.insert(1, os.path.join(sys.path[0], '..'))
%load_ext autoreload
%autoreload 2



from snorkel.snorkel import *
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (18,6)
filename = "text/AGR2_blood_biomarker.txt"

In [ ]:
Relations = BiomarkerDiseaseRelation_Extraction.generateRelations(_filename= filename)

In [ ]:
#NOTE: Relations.pkl is probably wrong file for this learning file
feats = None
relationsFile = "cache/" + filename + "/relations.pkl"
if not os.path.exists("cache/" + filename +"/" ):
    os.makedirs("cache/" + filename + "/")
try:
    with open(relationsFile, 'rb') as f:
        feats = cPickle.load(f)
except:
    %time Relations.extract_features()
    with open(relationsFile, 'w+') as f:
        cPickle.dump(Relations.feats, f)

DDL = DDLiteModel(Relations, feats)
print "Extracted {} features for each of {} mentions".format(DDL.num_feats(), DDL.num_candidates())

In [ ]:
tuples = []
list = re.split("[^\\S ]", open("tags6.tsv").read())
count = 7
while count < len(list):
    number = 0
    if (list[count + 6] == "true"):
        number = 1
    elif (list[count + 6] == "false"):
        number = -1
    tuples.append((list[count + 5] + "::" + list[count + 3] + "::[" + list[count + 4] + ", " + list[count] + "]::['" +
                 list[count + 1] + "', '" + list[count + 2] + "']", number))
    count += 7
gt = []
uids = []
for tuple in tuples:
    uids.append(tuple[0])
    gt.append(tuple[1])


# with open('examples/gene_tag_example/gt/uids.pkl', 'rb') as f:
#     uids2 = cPickle.load(f)
# with open('examples/gene_tag_example/gt/gt.pkl', 'rb') as f:
#     gt2 = cPickle.load(f)
    
DDL.update_gt(gt = gt[:50], uids = uids[:50])
DDL.set_holdout(validation_frac=0.5)

In [ ]:
DDL.open_mindtagger(num_sample=200, width='100%', height=1200)

In [ ]:
DDL.add_mindtagger_tags()
DDL.update_gt(gt=gt[50:], uids=uids[50:])

In [ ]:
sentence_keyword_lemmas = ["contain", "collect", "find", "sample", "fluid", "tissue", "detection"]
sentences = []
# 1- distance far
def LF_distance_far(m):
    # print m.lemmas
    # print m.dep_labels
    distance = abs(m.e2_idxs[0] - m.e1_idxs[0])
    if distance < 10:
        return 0
    else:
        return -1
        
            
# 2- distance close
def LF_distance_close(m):
    # print m.lemmas
    # print m.dep_labels
    distance = abs(m.e2_idxs[0] - m.e1_idxs[0])
    if distance < 5:
        return 1
    else:
        return 0

# 3-  Biomarker + preposition + Medium
def LF_IN(m):
    return 1 if (('IN' in m.post_window1('poses', 5) and 'IN' in m.pre_window2('poses',5))) or (('IN' in m.post_window2('poses', 5) and 'IN' in m.pre_window1('poses',5))) else 0 
         
# 4- If the sentence contains common keywords
def LF_keyword(m):
    for word in sentence_keyword_lemmas:
        if (word in m.pre_window1('lemmas',20)) or (word in m.post_window1('lemmas',20)):
            #if presenceOfNot(m):
            #    return -1
            #else:
            #    return 1
            return 1
    return 0

# 5- Medium "from" patients or subjects
def LF_From(m):
    index = 0
    post = m.post_window2('lemmas',20)
    if 'patient' in post:
        index = post.index('patient')
    elif 'subject' in post:
        index = post.index('subject')
    return 1 if "from" in m.post_window2('lemmas',index) else 0

# 6- Medium "-based": blood-based biomarker
def LF_based(m):
    mediumBased = m.mention2(attribute='words') + "-based"
    return 1 if mediumBased in m.pre_window1('words',20) or mediumBased in m.post_window1('words',20) else 0

# 7- Medium "biomarker": blood biomarker
def LF_biomarker(m):
    return 1 if 'biomarker' in m.post_window2('lemmas',3) else 0

# 8- if that relationship is in the references, MIGHT NOT WORK
def LF_References(m):
    sent_id = m.post_window1('sent_id',1)
    sentences_before = sentences[0:sent_id]
    for sentence in sentences_before:
        if 'References' in sentence.words():
            return -1
    return 0

In [ ]:
LFs = []
DDL.apply_lfs(LFs, clear=False)

In [ ]:
DDL.lowest_empirical_accuracy_lfs(n=10)

In [ ]:
matplotlib.rcParams['figure.figsize'] = (12,4)
mu_seq = np.ravel([1e-9, 1e-5, 1e-3, 1e-1])
lf_opts = {'sample': False, 'verbose': True}
model_opts = {'sample': False, 'n_iter': 3000, 'alpha': 0.5, 'mu': mu_seq, 'bias': False, 'verbose': True}
%time DDL.train_model(method='lr', lf_opts=lf_opts, model_opts=model_opts)

In [ ]:
DDL.plot_calibration()

In [ ]:
idxs, gt = DDL.get_labeled_ground_truth(subset=DDL.holdout())
acc_feats = np.mean(DDL.get_predicted(subset=DDL.holdout()) == gt)
acc_lfs = np.mean(DDL.get_lf_predicted(subset=DDL.holdout()) == gt)
print "LF accuracy: {:.3f}\nFull model accuracy: {:.3f}".format(acc_lfs, acc_feats)